In [20]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
import os
#Importing necessary libraries

In [21]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 48, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, low_level_features):
        low_level_features = self.conv1(low_level_features)
        low_level_features = self.bn1(low_level_features)
        low_level_features = self.relu(low_level_features)
        
        x = F.interpolate(x, size=low_level_features.size()[2:], mode='bilinear', align_corners=False)
        x = torch.cat((x, low_level_features), dim=1)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        
        return x


In [23]:
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ASPP, self).__init__()
        # Define dilated convolutions with different dilation rates
        self.conv1x1_1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3x3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3x3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.conv3x3_18 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=18, dilation=18)

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Convolution to adjust dimensions
        self.conv1x1_adjust = nn.Conv2d(in_channels * 4, out_channels, kernel_size=1)

    def forward(self, x):
        # Apply dilated convolutions with different dilation rates
        out_1x1 = self.conv1x1_1(x)
        out_3x3_6 = self.conv3x3_6(x)
        out_3x3_12 = self.conv3x3_12(x)
        out_3x3_18 = self.conv3x3_18(x)

        # Global average pooling
        global_pool = self.global_avg_pool(x)
        global_pool = torch.cat([global_pool] * x.size()[2:], dim=2)
        global_pool = torch.cat([global_pool] * x.size()[3:], dim=3)

        # Concatenate the outputs of all paths
        out_concat = torch.cat([out_1x1, out_3x3_6, out_3x3_12, out_3x3_18, global_pool], dim=1)

        # Apply 1x1 convolution to adjust dimensions
        out_adjust = self.conv1x1_adjust(out_concat)

        return out_adjust

In [24]:
class DeepLabV3Plus(nn.Module):
    def __init__(self, num_classes):
        super(DeepLabV3Plus, self).__init__()
        # Define the backbone (e.g., ResNet)
        self.backbone = models.resnet50(pretrained=False)
        # Modify the backbone's final layers to match the output stride required for DeepLabv3+
        self.backbone.layer4[0].conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
        # Adjust the strides or use dilated convolutions to achieve the desired output stride
        # Define ASPP module
        self.aspp = ASPP(2048, 256)
        # Define decoder module
        self.decoder = Decoder(256, num_classes)
        # Upsample the output
        self.upsample = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)
        # Final convolution to get the segmentation map
        self.final_conv = nn.Conv2d(256, num_classes, kernel_size=1)
        
    def forward(self, x):
        # Implement the forward pass
        x_size = x.size()
        features = self.backbone(x)
        aspp_output = self.aspp(features['out'])
        decoder_output = self.decoder(aspp_output, features['low_level'])
        upsampled_output = self.upsample(decoder_output)
        segmentation_map = self.final_conv(upsampled_output)
        return segmentation_map

In [27]:
# Define the path to the directory containing your images
root_dir = "../Train"

# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to 256x256 pixels
    transforms.ToTensor()            # Convert the image to a PyTorch tensor
])

# Create an instance of the ImageDataset class
dataset = ImageDataset(root_dir, transform=transform)

# Optionally, you can also use PyTorch's ImageFolder class which automatically loads images from subfolders
# dataset = ImageFolder(root=root_dir, transform=transform)

# Create a DataLoader to iterate over the dataset
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



FileNotFoundError: [Errno 2] No such file or directory: '../Train'

In [ ]:
num_classes = 4
model = DeepLabV3Plus(num_classes)
# Define your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define your loss function
criterion = nn.CrossEntropyLoss()

/Users/thomasstewart/micromamba/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/thomasstewart/micromamba/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
